In [1]:
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

import pandas as pd

In [4]:
q = '''select * from ga4_traffic limit 4'''
redshift_query_read(q, schema='production')

,date,org,form,views,sessions,sessionduration,bounces,path,devicecategory,qgiv_frontend,p2p_frontend,bouncerate
0,2023-05-02,0,0,1,1,199.0,0,/account/,mobile,False,False,0.0
1,2023-05-02,0,0,2,0,0.0,0,/account/1048262/,mobile,False,False,0.0
2,2023-05-02,0,0,1,0,0.0,0,/account/1089163/reset/97418aaa456d359510a1de9...,mobile,False,False,0.0
3,2023-05-02,0,0,1,0,0.0,0,/account/1159413/recurring/,desktop,False,False,0.0


In [5]:
q = '''select
            date_trunc('month', date) as month,
            devicecategory as platform,
            sum(views) as views
        from ga4_traffic
        group by date_trunc('month', date), devicecategory'''
traff = redshift_query_read(q, schema='production')

In [8]:
traff = traff.pivot(index='month', columns='platform', values='views').reset_index()

In [10]:
traff.drop('smart tv', axis=1, inplace=True)
traff.sort_values('month', ascending=True)

platform,month,desktop,mobile,tablet
0,2023-01-01,1289076.0,1267158.0,43095.0
1,2023-02-01,1810365.0,2040862.0,58344.0
2,2023-03-01,2210047.0,2198134.0,59901.0
3,2023-04-01,2043759.0,2165458.0,57761.0
4,2023-05-01,1555188.0,1765575.0,50426.0
5,2023-06-01,1144105.0,1268298.0,42334.0
6,2023-07-01,1091775.0,1190376.0,35697.0
7,2023-08-01,2010946.0,1664701.0,51727.0
8,2023-09-01,2599530.0,2180595.0,24677.0
9,2023-10-01,2717289.0,2384755.0,28067.0


In [11]:
cols = ['desktop', 'mobile', 'tablet']
traff['desktop_perc'] = traff['desktop'] / traff[cols].sum(axis=1)
traff['mobile_perc'] = traff['mobile'] / traff[cols].sum(axis=1)
traff['tablet_perc'] = traff['tablet'] / traff[cols].sum(axis=1)

In [12]:
traff

platform,month,desktop,mobile,tablet,desktop_perc,mobile_perc,tablet_perc
0,2023-01-01,1289076.0,1267158.0,43095.0,0.495926,0.487494,0.016579
1,2023-02-01,1810365.0,2040862.0,58344.0,0.463060,0.522017,0.014923
2,2023-03-01,2210047.0,2198134.0,59901.0,0.494630,0.491964,0.013406
3,2023-04-01,2043759.0,2165458.0,57761.0,0.478971,0.507492,0.013537
4,2023-05-01,1555188.0,1765575.0,50426.0,0.461317,0.523725,0.014958
5,2023-06-01,1144105.0,1268298.0,42334.0,0.466080,0.516674,0.017246
6,2023-07-01,1091775.0,1190376.0,35697.0,0.471030,0.513569,0.015401
7,2023-08-01,2010946.0,1664701.0,51727.0,0.539507,0.446615,0.013878
8,2023-09-01,2599530.0,2180595.0,24677.0,0.541027,0.453837,0.005136
9,2023-10-01,2717289.0,2384755.0,28067.0,0.529675,0.464854,0.005471


In [16]:
traff['all'] = traff[cols].sum(axis=1)
traff['desktop'].sum() / traff['all'].sum()

0.5051097719463521